In [93]:
import os
import pandas as pd
import numpy as np 

interesting directories: 
Other
Physical Activity 
Sleep 
Stress

Things to explore:
1. how to save the data between uses (so I don't have to read from file every time) 

    pickling? 
    creating a csv? 
    creating a sql database to pull in? 

To do: 
(1) 
keep tables separate until I get a daily summary for each activity 
Tables:
-bpm 
-sleep levels 
-sleep score (done)
-steps
-sedentary minutes 
-temperature 
-etc.?
merge all the tables together using date as the primary key 
daily sleep will be the response variabel 
everything else will be explanatory variables 

(2) create a function that takes in the file name and the folder and imports all of the data 

In [17]:
path = "/Users/jackiekinsler/projects/sleep_analysis_py/physical_data"
 
# to store metric types in a set
unique_metrics = set()

# dirs=directories
for (root, dirs, file) in os.walk(path):
    for f in file:
        if f.endswith(".csv"):
            metric = f.split("-")[0]
            unique_metrics.add(metric)
        
for metric in unique_metrics:
    print(metric)

Respiratory Rate Summary 
Stress Score.csv
Daily SpO2 
Computed Temperature 
sleep_score.csv
Daily Respiratory Rate Summary 
estimated_oxygen_variation
Active Zone Minutes 
Daily Heart Rate Variability Summary 
Device Temperature 
Heart Rate Variability Histogram 
Heart Rate Variability Details 


In [108]:
# this will streamline importing the many tables from the raw Fitbit JSON 
# Takes a string and returns a dataframe of the concatenated data from the JSON files 
def import_data_from_dir(file_prefix, directory):
    dfs = []
    for file in os.listdir(directory):
        if file_prefix in file: 
            print(file)
            dfs.append(pd.read_json(f"{directory}/{file}"))
    return pd.concat(dfs)

In [113]:
# Importing BPM data [dateTime, bpm, confidence] recorded every 5 seconds 
# /Users/jackiekinsler/projects/sleep_analysis_py/JacklynKinsler/Physical_Activity

bpm_nested = import_data_from_dir('heart_rate-', '/Users/jackiekinsler/projects/sleep_analysis_py/test_dir')
# bpm_nested has a 'value' column nested data as follows {value: {bpm: <int>, confidence: <int>}}
# Next line will separate the nested data into two columns
bpm = pd.concat([bpm_nested.drop(['value'], axis=1), bpm_nested['value'].apply(pd.Series)], axis=1)
bpm.head()

heart_rate-2017-08-10.json
heart_rate-2018-01-13.json
heart_rate-2018-01-12.json
heart_rate-2018-01-14.json


,dateTime,bpm,confidence
0,2017-08-10 11:17:39,70,0
1,2017-08-10 11:17:45,70,0
2,2017-08-10 11:17:51,70,0
3,2017-08-10 11:17:57,70,0
4,2017-08-10 11:18:03,70,0


From the Fitbit community: 
“Confidence scores indicate confidence in the accuracy of the captured data with “3” being the highest and “0” being the lowest; a score of 0 indicates no Heart Rate is detected by the Fitbit tracker and no Heart Rate is displayed on the tracker device”.

In [115]:
# Importing detailed sleep data 
df_sleep_nested = import_data_from_dir('sleep-', '/Users/jackiekinsler/projects/sleep_analysis_py/test_dir')

# df = pd.concat([df_nested.drop(['value'], axis=1), df_nested['value'].apply(pd.Series)], axis=1)
df_sleep_nested.head()
df_sleep_nested.loc[0,'levels']
# the summary of the sleep data seems to have everything interesting (cumulative data)
# the rest of the levels column is super detailed information about each type and duration of sleep segment 

sleep-2021-04-28.json
sleep-2018-07-13.json


0    {'summary': {'deep': {'count': 7, 'minutes': 7...
0    {'summary': {'deep': {'count': 5, 'minutes': 9...
Name: levels, dtype: object

In [69]:
df_sleep_nested.head()

,logId,dateOfSleep,startTime,endTime,duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,timeInBed,efficiency,type,infoCode,logType,levels,mainSleep
0,19156244739,2018-08-12,2018-08-11T23:58:00.000,2018-08-12T08:04:30.000,29160000,0,428,58,0,486,91,stages,0,auto_detected,"{'summary': {'deep': {'count': 5, 'minutes': 9...",True
1,19146597672,2018-08-11,2018-08-10T23:06:00.000,2018-08-11T08:19:00.000,33180000,0,455,98,0,553,88,stages,0,auto_detected,"{'summary': {'deep': {'count': 7, 'minutes': 1...",True
2,19140164672,2018-08-10,2018-08-09T23:23:00.000,2018-08-10T08:38:30.000,33300000,0,473,82,0,555,90,stages,0,auto_detected,"{'summary': {'deep': {'count': 7, 'minutes': 1...",True
3,19129203312,2018-08-09,2018-08-08T23:34:00.000,2018-08-09T07:35:00.000,28860000,0,414,67,0,481,96,stages,0,auto_detected,"{'summary': {'deep': {'count': 6, 'minutes': 8...",True
4,19122684524,2018-08-08,2018-08-07T22:07:00.000,2018-08-08T06:00:00.000,28380000,0,419,54,0,473,94,stages,0,auto_detected,"{'summary': {'deep': {'count': 5, 'minutes': 9...",True


In [67]:
df_sleep_nested['dateOfSleep'].is_unique
df_sleep_nested['dateOfSleep'].duplicated
# below I'm grabbing the data from a night that had two sleep entries (aka a duplicate date)
df_sleep_nested.loc[df_sleep_nested['dateOfSleep'] == '2018-07-31']

,logId,dateOfSleep,startTime,endTime,duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,timeInBed,efficiency,type,infoCode,logType,levels,mainSleep
11,19036267747,2018-07-31,2018-07-31T06:07:00.000,2018-07-31T07:41:00.000,5640000,0,85,9,0,94,90,classic,2,auto_detected,"{'summary': {'restless': {'count': 4, 'minutes...",False
12,19033727531,2018-07-31,2018-07-30T23:35:00.000,2018-07-31T05:01:30.000,19560000,0,282,44,4,326,92,stages,0,auto_detected,"{'summary': {'deep': {'count': 4, 'minutes': 6...",True


sleep score in fitbit: 
source: https://help.fitbit.com/articles/en_US/Help_article/2439.htm



Note: shortData is only included for stages sleep logs and includes wake periods that are 3 minutes or less in duration. This distinction is to simplify graphically distinguishing short wakes from longer wakes, but they are physiologically equivalent.

from: https://dev.fitbit.com/build/reference/web-api/sleep/get-sleep-log-by-date-range/#:~:text=Note%3A%20shortData%20is%20only%20included,but%20they%20are%20physiologically%20equivalent.

sleep score is in a .csv with seemingly 1 row per day 

In [72]:
# /Users/jackiekinsler/projects/sleep_analysis_py/JacklynKinsler/Sleep/sleep_score.csv
sleep_score = pd.read_csv('/Users/jackiekinsler/projects/sleep_analysis_py/JacklynKinsler/Sleep/sleep_score.csv')
sleep_score.head()

,sleep_log_entry_id,timestamp,overall_score,composition_score,revitalization_score,duration_score,deep_sleep_in_minutes,resting_heart_rate,restlessness
0,39497287745,2022-12-19T11:00:30Z,86,22,22,42,102,53,0.080976
1,39483974879,2022-12-18T07:17:00Z,84,21,21,42,105,53,0.080969
2,39483974878,2022-12-17T02:48:30Z,61,15,18,28,53,52,0.095041
3,39457704744,2022-12-16T04:45:00Z,67,17,19,31,74,52,0.103937
4,39447333347,2022-12-15T08:07:30Z,77,19,21,37,77,51,0.093642


import steps data 
steps data is recorded every few minutes, and it records the steps in that period of time 
will reduce the data to get the total steps for each day 

In [78]:
# /Users/jackiekinsler/projects/sleep_analysis_py/test_dir
dfs = []
# for file in os.listdir("/Users/jackiekinsler/projects/sleep_analysis_py/JacklynKinsler/Physical_Activity"):
for file in os.listdir("/Users/jackiekinsler/projects/sleep_analysis_py/test_dir"):
    if "steps-" in file: 
        print(file)
#         dfs.append(pd.read_json(f"/Users/jackiekinsler/projects/sleep_analysis_py/JacklynKinsler/Physical_Activity/{file}"))
        dfs.append(pd.read_json(f"/Users/jackiekinsler/projects/sleep_analysis_py/test_dir/{file}"))
# df = pd.concat(dfs)

step_detail = pd.concat(dfs)

# df = pd.concat([df_nested.drop(['value'], axis=1), df_nested['value'].apply(pd.Series)], axis=1)
step_detail.head()

steps-2017-11-15.json
steps-2017-09-16.json
steps-2017-10-16.json


,dateTime,value
0,2017-11-15 05:00:00,0
1,2017-11-15 05:01:00,0
2,2017-11-15 05:02:00,0
3,2017-11-15 05:03:00,0
4,2017-11-15 05:04:00,0


In [81]:
step_detail.loc[step_detail['value'] != 0]

,dateTime,value
428,2017-11-16 09:41:00,9
519,2017-11-16 11:12:00,23
521,2017-11-16 11:14:00,27
522,2017-11-16 11:15:00,18
523,2017-11-16 11:16:00,4
...,...,...
39133,2017-11-15 03:08:00,31
39140,2017-11-15 03:15:00,4
39145,2017-11-15 03:20:00,7
39159,2017-11-15 03:34:00,14


In [97]:
sleep_example = import_data_from_dir('tab-', '/Users/jackiekinsler/projects/sleep_analysis_py/test_dir')
sleep_example.head()

ValueError: A very specific bad thing happened